In [ ]:
# pip install google-generativeai

In [1]:
import google.generativeai as genai

In [2]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML

In [171]:
import json
import time

import re
import ast

import os

In [4]:
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('laptop_data.csv')
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [5]:
df.Description

0     The Dell Inspiron is a versatile laptop that c...
1     The MSI GL65 is a high-performance laptop desi...
2     The HP EliteBook is a premium laptop designed ...
3     The Lenovo IdeaPad is a versatile laptop that ...
4     The ASUS ZenBook Pro is a high-end laptop that...
5     The Acer Predator is a powerhouse laptop desig...
6     The Microsoft Surface Laptop is a premium devi...
7     The Lenovo ThinkPad is a powerful laptop desig...
8     The HP Pavilion is a budget-friendly laptop th...
9     The ASUS ROG Strix G is a high-performance gam...
10    The Dell XPS 15 is a premium laptop that combi...
11    The Lenovo ThinkPad X1 Carbon is a sleek and l...
12    The Acer Swift 3 is a lightweight and affordab...
13    The Apple MacBook Air is a sleek and lightweig...
14    The MSI Prestige 14 is a compact and stylish l...
15    The ASUS ZenBook 13 is a lightweight and power...
16    The Dell Precision 5550 is a high-performance ...
17    The HP ENVY x360 is a versatile 2-in-1 con

<img src="stage_system_design.png"/>

# Stage 1: Intent Clarity and Intent Confirmation

<img src="image+3.jpg"/>

In [6]:
from tenacity import retry, wait_random_exponential, stop_after_attempt
import asyncio

In [7]:
GOOGLE_API_KEY = open("gemini_API_Key.txt", "r").read().strip()
genai.configure(api_key=GOOGLE_API_KEY)

## Step1: Initialise the conversation with base prompt
- Role of the model
- Guidelines for the model
- Instruction within the transcripts
- Chain of Thoughts
- Few Shot examples (samples)

In [8]:
def initialize_conversation():
    '''
    Returns a list [{"role": "model", "parts": [system_message]}]
    '''

    delimiter = "####"

    example_user_dict = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "low",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}

    example_user_req = {'GPU intensity': "_",
                        'Display quality': "_",
                        'Portability': "_",
                        'Multitasking': "_",
                        'Processing speed': "_",
                        'Budget': "_"}

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'Budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Model: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Model: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Model: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Model:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Model: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "model", "parts": [system_message]}]
    # conversation = system_message
    return conversation

#### Displaying the base prompt

In [9]:
debug_conversation = initialize_conversation()
print(debug_conversation[0]['parts'][0])


    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.


In [10]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

In [ ]:
# Moderation does not work well with Gemini
def moderation_check(user_input):
    MODEL = 'gemini-1.5-flash'
    model = genai.GenerativeModel(MODEL)
    response = model.generate_content(user_input, safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    })

    print(response)
    # for rating in response.candidates[0].safety_ratings:
        # if rating.HarmProbability > HarmCategory.:
        #     return False

    # return True
        

In [ ]:
moderation_check("I want to kill them")

### `get_chat_completions` - Define the function get_chat_completions with two parameters
- prompt
- Is Json response type

The prompt this time will contain the user query as well
- If json response type then we will return a json
- Else, we will return text result

`@retry` - In case of failure, retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay

In [168]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gemini-1.5-flash'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""
    gemini_model = genai.GenerativeModel(MODEL)
    # Only 1 candidate_count is allowed as of now
    if json_format:
        gen_config = genai.types.GenerationConfig(candidate_count=1,
                                              temperature=0, 
                                              response_mime_type="application/json")
        chat_completion_json = gemini_model.generate_content(input, 
                                           generation_config=gen_config)

        output = json.loads(chat_completion_json.candidates[0].content.parts[0].text)
        return output
    else:
        gen_config = genai.types.GenerationConfig(candidate_count=1,
                                              temperature=0)
        chat_completion_json = gemini_model.generate_content(input, 
                                           generation_config=gen_config)

        output = chat_completion_json.candidates[0].content.parts[0].text
        return output

In [ ]:
# Testing the OpenAI functions defined above
input_prompt ='What is the capital of France?'
messages = [{'role':'user','parts':[input_prompt]}]
messages

In [ ]:
get_chat_completions(messages), get_chat_completions(messages, json_format=True)

### `iterate_llm_response` - Iterate the same prompt for `n` number of times and verify the consistency of the result

In [12]:
def iterate_llm_response(funct, debug_response, num = 10):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer'
# iterate_llm_response(get_chat_completions, messages)

## Step2: Initialise the user query in the base prompt
- Initialise user query in base prompt
- Send it to the model to evaluate
- Model will return text output or json output based on the parameter sent to `get_chat_completion` function
- Add system response to the base prompt

#### Initialise user query in base prompt

In [13]:
debug_user_input = "Hi, I am Anand. I need a laptop for coding."

In [14]:
debug_conversation.append({"role": "user", "parts": [debug_user_input]})
print(debug_conversation[1]["parts"][0])

Hi, I am Anand. I need a laptop for coding.


In [15]:
display(debug_conversation)

[{'role': 'model',
  'parts': ['\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stat

#### Send it to the model to evaluate - Text response

In [17]:
debug_response_assistant = get_chat_completions(debug_conversation)
print(debug_response_assistant)

Hi Anand! That's great! Coding requires a laptop that can handle multiple programs running simultaneously and has a good processing speed.  To give you the best recommendations, could you tell me a bit more about your coding needs? 

For example:

* **What kind of coding do you do?** (Web development, data science, game development, etc.)
* **Do you need a powerful graphics card for your work?** (For example, if you're doing game development or machine learning with heavy graphics processing)
* **How important is portability to you?** (Do you need a laptop you can easily carry around, or will you be using it primarily at a desk?)
* **What is your budget for the laptop?** 

Once I have a better understanding of your requirements, I can help you find the perfect laptop for your coding needs. 



#### Send it to the model to evaluate - Json response

In [18]:
debug_response_assistant_json = get_chat_completions(debug_conversation, json_format=True)
print(debug_response_assistant_json)

{'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'medium', 'Budget': '50000'}


#### Add system response to the base prompt

In [19]:
debug_conversation.append({"role": "model", "parts": [debug_response_assistant]})

In [21]:
display(debug_conversation)

[{'role': 'model',
  'parts': ['\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stat

### intent_confirmation_layer()
This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks 

    Parameters:
    - response_assistant (str): The input text containing user requirements captured through 6 keys:
        'GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed', and 'Budget'.

    Returns:
    - str: A one-word string in JSON format indicating if the values for the specified keys are correctly filled.
        - 'Yes' if the values are correctly filled for all keys ('GPU intensity', 'Display quality', 'Portability',
          'Multitasking', 'Processing speed') based on the importance as stated by the user.
        - 'No' otherwise.

    Note:
    - The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on their importance as stated by the user.
    - The input text should be structured such that it contains the necessary keys and their corresponding values.
    - The function uses OpenAI's Chat Completion API to evaluate the correctness of the input values.

if the following properties for the user has been captured or not
   - GPU intensity
   - Display quality
   - Portability
   - Multitasking
   - Processing speed
   - Budget

In [200]:
def intent_confirmation_layer(response_assistant):
    MODEL = 'gemini-1.5-flash'
    gemini_model = genai.GenerativeModel(MODEL)
    
    delimiter = "####"
    allowed_values = {'low','medium','high'}
    
    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'GPU intensity': 'values',
    'Display quality':'values',
    'Portability':'values',
    'Multitasking':'values',
    'Processing speed':'values',
    'Budget':'number'
    }}
    The values for the keys should only be from the allowed values: {allowed_values}.
    The 'Budget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    Thought 3 - Think carefully before the answering.
    Thought 4 - If any of these 6 Keys are not filled then result No. Always verify before Yes, you should have all the keys
    """

    messages=[{"role": "model", "parts":[prompt] },
              {"role": "user", "parts":[f"""Here is the input: {response_assistant}"""]}]

    gen_config = genai.types.GenerationConfig(candidate_count=1,
                                              temperature=0, 
                                              response_mime_type="application/json")
    intent_confirmation_json = gemini_model.generate_content(messages, 
                                       generation_config=gen_config)

    output = json.loads(intent_confirmation_json.candidates[0].content.parts[0].text)

    return output

In [23]:
print(debug_response_assistant)

Hi Anand! That's great! Coding requires a laptop that can handle multiple programs running simultaneously and has a good processing speed.  To give you the best recommendations, could you tell me a bit more about your coding needs? 

For example:

* **What kind of coding do you do?** (Web development, data science, game development, etc.)
* **Do you need a powerful graphics card for your work?** (For example, if you're doing game development or machine learning with heavy graphics processing)
* **How important is portability to you?** (Do you need a laptop you can easily carry around, or will you be using it primarily at a desk?)
* **What is your budget for the laptop?** 

Once I have a better understanding of your requirements, I can help you find the perfect laptop for your coding needs. 



In [24]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
display(debug_confirmation)

{'result': 'No', 'reason': 'Missing keys'}

#### Extract the dictionary from the response

In [25]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'GPU intensity': 'high',
                'Display quality': 'high',
                'Portability': 'medium',
                'Multitasking': 'high',
                'Processing speed': 'high',
                'Budget': '200000'}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'GPU intensity': 'low/medium/high ',
            'Display quality':'low/medium/high',
            'Portability':'low/medium/high',
            'Multitasking':'low/medium/high',
            'Processing speed':'low/medium/high',
            'Budget':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            """
    messages = [{"role": "model", "parts":[prompt]},
                {"role": "user", "parts":[f"""Here is the user input: {response}""" ]}]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

In [29]:
debug_response_assistant_n = f"""Thank you for providing your budget.
Based on your budget of 80,000 INR, I will consider this while recommending suitable laptop options for you.
Here is the final recommendation for your laptop:
- GPU intensity: high
- Display quality: high
- Portability: low
- Multitasking: high
- Processing speed: medium
- Budget: 80,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget.
Let me know if there's anything else I can assist you with!"""

In [ ]:
iterate_llm_response(dictionary_present, debug_response_assistant_n)

In [30]:
debug_conversation = initialize_conversation()
debug_user_input = "Hi, I am Anand. I need a laptop for coding."
# debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "parts": [debug_user_input]})
debug_response_assistant = get_chat_completions(debug_conversation)
# debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "model", "parts": [debug_response_assistant]})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
# After a series of conversation...
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '80000'}


# Stage 2: Product Extraction and Product Mapping

<img src="Stage+2.jpg"/>

### product_map_layer()
This function is responsible for extracting key features and criteria from laptop descriptions. Here's a breakdown of how it works:
-  Use a prompt that assign it the role of a Laptop Specifications Classifier, whose objective is to extract key features and classify them based on laptop descriptions.
- Provide step-by-step instructions for extracting laptop features from description.
- Assign specific rules for each feature (e.g., GPU Intensity, Display Quality, Portability, Multitasking, Processing Speed) and associate them with the appropriate classification value (Low, Medium, or High).
- Includes Few Shot Prompting (sample conversation between the user and assistant) to demonstrate the expected result of the feature extraction and classification process.

#### Extracts key features from a laptop description and classifies them based on predefined rules.
- Args: laptop_description (str): The description of the laptop to be analyzed.
- Returns:
        - dict: A dictionary containing the classified features of the laptop.
            Keys represent different aspects such as GPU intensity, Display quality, Portability,
            Multitasking, and Processing speed, each mapped to a value indicating the classification
            (low, medium, or high) based on the extracted feature

In [31]:
def product_map_layer(laptop_description):
    delimiter = "#####"
    lap_spec = {
        "GPU intensity":"(Type of the Graphics Processor)",
        "Display quality":"(Screen Resolution)",
        "Portability":"(Laptop Weight)",
        "Multitasking":"(RAM)",
        "Processing speed":"(CPU Manufacturer, Core)"
    }
    values = {'low','medium','high'}

    prompt=f"""
            You are a Laptop Specifications Classifier whose job is to extract the key features of laptops as per their requirements.
            To analyze each laptop, perform the following steps:
            Step 1: Extract the laptop's primary features from the description {laptop_description}
            Step 2: Store the extracted features in {lap_spec} \
            Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
            
            {delimiter}
            GPU Intensity: low: < for entry-level graphics like Intel UHD > , \n
            medium: < if Mid-range dedicated graphics like M1, AMD Radeon, Intel Iris > , \n
            high: < High-end dedicated graphics like Nvidia, M2, M3 > , \n


            Display Quality: low: < if resolution below Full HD (e.g., 1366x768). > , \n
            medium: < if Full HD resolution (1920x1080) > , \n
            high: < if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. > \n


            Portability: low: < if laptop weight is greater than 2.51 kg > , \n
            medium: < if laptop weight is between 1.51 kg and 2.51 kg> , \n
            high: < if laptop weight is less than 1.51 kg> \n


            Multitasking: low: < If the value of the RAM is 8GB or lower> , \n
            medium: < if the value of the RAM is between 8GB & 16GB > , \n
            high: < if the value of the RAM is more than 32GB> \n


            Processing Speed: low: < if entry-level processors like Intel Core i3, AMD Ryzen 3 > , \n
            medium: < if Mid-range processors like Intel Core i5, AMD Ryzen 5, M1 > , \n
            high: < if High-performance processors like M2, Intel Core i7, AMD Ryzen 7 or higher > \n
            {delimiter}


            {delimiter}
            Here are some input output pairs for few-shot learning:
            input1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring basic multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is decently portable. Additionally, it comes with an Intel UHD GPU for basic graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. "
            output1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'low','Processing speed':'medium'}}


            input2: "The Lenovo ThinkPad X1 Carbon is a sleek and lightweight laptop designed for professionals on the go. It is equipped with an Intel Core i7 processor running at 2.6 GHz, providing strong processing capabilities for productivity. With 16GB of RAM and an SSD, it offers decent multitasking performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It comes with Intel UHD integrated graphics for basic graphical performance. Weighing just 1.13 kg, it is extremely lightweight and highly portable. The laptop features an IR camera for face unlock, providing convenient and secure login options. With a three-year warranty and an impressive battery life of up to 12 hours, the Lenovo ThinkPad X1 Carbon ensures reliability and long-lasting productivity. "
            output2: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'high', 'Multitasking':'medium', 'Processing speed':'high'}}


            input3: "The Apple MacBook Pro is a high-end laptop that combines top-tier performance with a stunning display. It is equipped with an M2 processor running at 2.9 GHz, providing exceptional processing power for demanding tasks and content creation. With 32GB of RAM and an SSD, it offers seamless multitasking and fast storage access for large projects. The laptop features a 16" Retina display with a resolution of 3072x1920, delivering breathtaking visuals and precise color reproduction. It comes with an Apple M2 dedicated graphics, ensuring smooth graphics performance for professional applications. Weighing 2.02 kg, it offers decent portability. The laptop features a True Tone display, adjusting the color temperature to match the ambient lighting for a more natural viewing experience. With a three-year warranty and a battery life of up to 10 hours, the Apple MacBook Pro offers reliability and endurance for professionals."
            output3: {{'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'medium','Multitasking': 'high', 'Processing speed': 'high'}}
            {delimiter}
            """

    input = f"""Follow the prompt instructions step-by-step and output the dictionary in JSON format for the following laptop {laptop_description}."""

    messages=[{"role": "model", "parts":[prompt]},{"role": "user","parts":[input]}]

    response = get_chat_completions(messages)

    return response

In [32]:
debug_laptop_description = f"""
The Dell Inspiron is a versatile laptop that combines powerful performance and affordability.
It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing.
With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity.
The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience.
Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage.
Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience.
With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment.
All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.
"""
display(product_map_layer(debug_laptop_description))
print(type(product_map_layer(debug_laptop_description)))

'```json\n{"GPU intensity": "low", "Display quality": "medium", "Portability": "medium", "Multitasking": "low", "Processing speed": "medium"}\n``` \n'

<class 'str'>


In [ ]:
iterate_llm_response(product_map_layer, debug_laptop_description)

### compare_laptops_with_user()
This function compares the user's profile with the different laptops and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input
    - Filter the laptops based on their price, keeping only the ones within the user's budget.
    - Calculate a score for each laptop based on how well it matches the user's requirements.
    - Sort the laptops based on their scores in descending order.
    - Return the top 3 laptops as a JSON-formatted string.

#### Compares laptops based on user requirements and returns the top 3 matching laptops in JSON format.

    Parameters:
    - user_req_string (str): A string representing user requirements in JSON format.

    Returns:
    - str: A JSON string containing information about the top 3 matching laptops based on user requirements.

### Update the Laptop CSV with the Laptop Feature
- Read each row of the Laptop database and apply product_map_layer
- This will provide a dictionary, store it in laptop feature function
- Use this new column for comparing with User requirements

In [177]:
def generate_dictionary_with_wait(columnSeries, isDebug):
    updated_features = []
    for item in columnSeries:
        time.sleep(10)
        result = product_map_layer(item)
        updated_features.append(result)
        if isDebug:
            print(result)
    return updated_features
def update_laptop_csv_with_feature(isDebug = False):
    laptop_csv = pd.read_csv("laptop_data.csv")
    updated_features = generate_dictionary_with_wait(laptop_csv["Description"], isDebug=isDebug)
    laptop_csv["laptop_feature"] = updated_features
    laptop_csv.to_csv("updated_laptop_data.csv")

In [ ]:
update_laptop_csv_with_feature()

In [34]:
updated_laptop_csv = pd.read_csv("updated_laptop_data.csv")
updated_laptop_csv

,Unnamed: 0,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"```json\n{""GPU intensity"": ""low"", ""Display qua..."
1,1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"```json\n{""GPU intensity"": ""high"", ""Display qu..."
2,2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"```json\n{""GPU intensity"": ""low"", ""Display qua..."
3,3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"```json\n{""GPU intensity"": ""low"", ""Display qua..."
4,4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"```json\n{""GPU intensity"": ""high"", ""Display qu..."
5,5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,"80,000",The Acer Predator is a powerhouse laptop desig...,"```json\n{""GPU intensity"": ""high"", ""Display qu..."
6,6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.3 kg,Touchscreen Display,1 year,10 hours,"90,000",The Microsoft Surface Laptop is a premium devi...,"```json\n{""GPU intensity"": ""medium"", ""Display ..."
7,7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,"60,000",The Lenovo ThinkPad is a powerful laptop desig...,"```json\n{""GPU intensity"": ""high"", ""Display qu..."
8,8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.1 kg,B&O Audio,1 year,4 hours,"30,000",The HP Pavilion is a budget-friendly laptop th...,"```json\n{""GPU intensity"": ""low"", ""Display qua..."
9,9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.9 kg,Aura Sync RGB Keyboard,2 years,5 hours,"85,000",The ASUS ROG Strix G is a high-performance gam...,"```json\n{""GPU intensity"": ""high"", ""Display qu..."


In [35]:
def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"


    dictionary_matches = re.findall(regex_pattern, string)


    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()


        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

print(updated_laptop_csv["laptop_feature"].apply(lambda x: extract_dictionary_from_string(x)))

0     {'gpu intensity': 'low', 'display quality': 'm...
1     {'gpu intensity': 'high', 'display quality': '...
2     {'gpu intensity': 'low', 'display quality': 'm...
3     {'gpu intensity': 'low', 'display quality': 'l...
4     {'gpu intensity': 'high', 'display quality': '...
5     {'gpu intensity': 'high', 'display quality': '...
6     {'gpu intensity': 'medium', 'display quality':...
7     {'gpu intensity': 'high', 'display quality': '...
8     {'gpu intensity': 'low', 'display quality': 'l...
9     {'gpu intensity': 'high', 'display quality': '...
10    {'gpu intensity': 'high', 'display quality': '...
11    {'gpu intensity': 'low', 'display quality': 'h...
12    {'gpu intensity': 'medium', 'display quality':...
13    {'gpu intensity': 'medium', 'display quality':...
14    {'gpu intensity': 'medium', 'display quality':...
15    {'gpu intensity': 'medium', 'display quality':...
16    {'gpu intensity': 'high', 'display quality': '...
17    {'gpu intensity': 'medium', 'display quali

In [37]:
def get_budget_in_int(requirements_dic):
    return requirements_dic.get('budget'.capitalize(), '0') if type(requirements_dic.get('budget'.capitalize(), '0')) == int else int(requirements_dic.get('budget'.capitalize(), '0').replace(',', '').split()[0])
    
def compare_laptops_with_user(user_req_string):
    laptop_df= pd.read_csv('updated_laptop_data.csv')
    user_requirements = user_req_string if type(user_req_string) == dict else extract_dictionary_from_string(user_req_string)
    budget =  get_budget_in_int(user_requirements)

    filtered_laptops = laptop_df.copy()
    filtered_laptops["Price"] = filtered_laptops["Price"].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops["Price"] <= budget].copy()
    
    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0


        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key.lower(), None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1


        filtered_laptops.loc[index, 'Score'] = score


    # Sort the laptops by score in descending order and return the top 5 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
    
    return top_laptops

In [42]:
compare_laptops_with_user(response_dict_n)

,Unnamed: 0,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,Score
7,7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,60000,The Lenovo ThinkPad is a powerful laptop desig...,4
1,1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,55000,The MSI GL65 is a high-performance laptop desi...,3
5,5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,80000,The Acer Predator is a powerhouse laptop desig...,3


In [90]:
def product_validation(recommendation):
    # recommendation = recommendation.drop("Unnamed", axis=1)
    validated_laptops = []
    for index, row in recommendation.iterrows():
        if row.Score > 2:
            validated_laptops.append(row)

    df = pd.concat(validated_laptops, axis=1).transpose()
    return df

In [91]:
product_validation(compare_laptops_with_user(response_dict_n))

,Unnamed: 0,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,Score
7,7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,60000,The Lenovo ThinkPad is a powerful laptop desig...,4
1,1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,55000,The MSI GL65 is a high-performance laptop desi...,3
5,5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,80000,The Acer Predator is a powerhouse laptop desig...,3


# Stage 3: Product Recommendation

<img src="Stage+3.jpg"/>

In [153]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue:.{products}\
    You should keep the user profile in mind while answering the questions.\


    Start with a brief summary of each laptop in the following format, in decreasing order of Score and if Score is same then arrange in the decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    Start with saying that "Here is the recommendation". Do not mention the order logic to the user
    """
    conversation = [{"role": "user", "parts": [system_message] }]
    return conversation

In [154]:
debug_recommendation_json = json.loads(product_validation(compare_laptops_with_user(response_dict_n)).to_json(orient="records"))

In [ ]:
print(list(map(lambda x: x["Description"], debug_recommendation_json)))

In [158]:
debug_conversation_reco = initialize_conv_reco(debug_recommendation_json)
debug_conversation_reco

[{'role': 'user',
  'parts': ['\n    You are an intelligent laptop gadget expert and you are tasked with the objective to     solve the user queries about any product from the catalogue:.[{\'Unnamed: 0\': 7, \'Brand\': \'Lenovo\', \'Model Name\': \'ThinkPad\', \'Core\': \'Ryzen 7\', \'CPU Manufacturer\': \'AMD\', \'Clock Speed\': \'3.0 GHz\', \'RAM Size\': \'16GB\', \'Storage Type\': \'SSD\', \'Display Type\': \'IPS\', \'Display Size\': \'14"\', \'Graphics Processor\': \'NVIDIA GTX\', \'Screen Resolution\': \'2560x1440\', \'OS\': \'Linux\', \'Laptop Weight\': \'1.6 kg\', \'Special Features\': \'Backlit Keyboard\', \'Warranty\': \'3 years\', \'Average Battery Life\': \'6 hours\', \'Price\': 60000, \'Description\': \'The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast st

In [155]:
# json_text = json.dumps(debug_recommendation_json, sort_keys=True, indent=4, separators=(',', ': '))
# json_text

'[\n    {\n        "Average Battery Life": "6 hours",\n        "Brand": "Lenovo",\n        "CPU Manufacturer": "AMD",\n        "Clock Speed": "3.0 GHz",\n        "Core": "Ryzen 7",\n        "Description": "The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast storage access. The laptop features a 14\\" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it is lightweight and highly portable. The laptop features a backlit keyboard for comfortable typing in low-light environments. With a three-year warranty and a battery life of up to 6 hours, the Lenovo ThinkPad offers reliability and durability. Priced at 60,000, it is an excellen

In [156]:
# debug_conversation_reco = initialize_conv_reco(json_text)
# debug_conversation_reco

[{'role': 'user',
  'parts': ['\n    You are an intelligent laptop gadget expert and you are tasked with the objective to     solve the user queries about any product from the catalogue:.[\n    {\n        "Average Battery Life": "6 hours",\n        "Brand": "Lenovo",\n        "CPU Manufacturer": "AMD",\n        "Clock Speed": "3.0 GHz",\n        "Core": "Ryzen 7",\n        "Description": "The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast storage access. The laptop features a 14\\" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It also comes with an NVIDIA GTX graphics card for enhanced graphical performance. Weighing just 1.6 kg, it is lightweight and highly portable. The laptop features a backlit keyboard for comfortable ty

In [161]:
debug_system_top3_recommendation = get_chat_completions(debug_conversation_reco)

In [163]:
debug_conversation_reco.append({"role":"model", "parts":[debug_system_top3_recommendation]})
debug_conversation_reco

[{'role': 'user',
  'parts': ['\n    You are an intelligent laptop gadget expert and you are tasked with the objective to     solve the user queries about any product from the catalogue:.[{\'Unnamed: 0\': 7, \'Brand\': \'Lenovo\', \'Model Name\': \'ThinkPad\', \'Core\': \'Ryzen 7\', \'CPU Manufacturer\': \'AMD\', \'Clock Speed\': \'3.0 GHz\', \'RAM Size\': \'16GB\', \'Storage Type\': \'SSD\', \'Display Type\': \'IPS\', \'Display Size\': \'14"\', \'Graphics Processor\': \'NVIDIA GTX\', \'Screen Resolution\': \'2560x1440\', \'OS\': \'Linux\', \'Laptop Weight\': \'1.6 kg\', \'Special Features\': \'Backlit Keyboard\', \'Warranty\': \'3 years\', \'Average Battery Life\': \'6 hours\', \'Price\': 60000, \'Description\': \'The Lenovo ThinkPad is a powerful laptop designed for professional users. It is equipped with a Ryzen 7 processor from AMD clocked at 3.0 GHz, providing strong processing capabilities for demanding tasks. With 16GB of RAM and an SSD, it offers smooth multitasking and fast st

In [ ]:
debug_conversation_reco.append({"role":"user", "parts":["Which of these is best suited for travel. Recommend only one"]})

In [165]:
final_result = get_chat_completions(debug_conversation_reco)
print(final_result)

For travel, the **Lenovo ThinkPad** is the best choice. It's the lightest of the three at 1.6kg, making it easier to carry around.  While the Acer Predator has a larger screen, its weight of 3.2kg would make it cumbersome for travel. 



In [174]:
os.path.isfile("updated_laptop_data.csv")

True

In [198]:
def check_moderation(user_input):
    return "Not Flagged" # moderation_check(user_input) # - Gemini does not support moderation on user input
    
def dialogue_mgmt_system(isDebug = False):

    print("Initialising system, please wait....\n")

    if os.path.isfile("updated_laptop_data.csv") == False:
        update_laptop_csv_with_feature()

    print("System is ready \n")
    
    conversation = initialize_conversation()
    print("--"*20 + "Shop Assitant" + "--"*20)
    print("""
    Hello, I am your laptop assistant. Please feel free to ask me about laptop recommendations. \n
    You can tell me about your needs like \n
    - what is the purpose of the laptop
    - Do you need heavy processing laptop
    - Do you need it for travel purposes, etc

    Please note: I do not take inputs to talk to about any specific brand as I am suppose to be non-biased. 
    Once I find a good recommendatio for you, then you can ask me about that laptop.\n
    Let's start!!!
    """ + '\n')
    top_3_laptops = None
    user_input = ''


    while(user_input != "exit"):
        print("--"*20 + "User" + "--"*20)
        user_input = input("")

        
        if check_moderation(user_input) == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break


        if top_3_laptops is None:
            conversation.append({"role": "user", "parts": [user_input]})


            response_assistant = get_chat_completions(conversation)

            if check_moderation(user_input) == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            if check_moderation(user_input) == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break


            if "No".lower() in json.dumps(confirmation, sort_keys=True, indent=4, separators=(',', ': ')).lower():
                conversation.append({"role": "model", "parts": [response_assistant]})
                print("--"*20 + "Shop Assitant" + "--"*20)
                print(f"\n{response_assistant}\n")
                
                if isDebug:
                    print(f"\n{confirmation}\n")
            else:
                print("--"*20 + "Shop Assitant" + "--"*20)
                print(f"\n{response_assistant}\n")
                
                if isDebug:
                    print(f"\n{confirmation}\n")
                    
                response = dictionary_present(response_assistant)

                if check_moderation(user_input) == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break




                print("--"*20 + "Shop Assitant" + "--"*20)
                
                if isDebug:
                    print(f'\n{response}\n')
                    
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptops_with_user(response)


                validated_reco = product_validation(top_3_laptops)


                if len(validated_reco) == 0:
                    print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
                    break


                conversation_reco = initialize_conv_reco(validated_reco)
                recommendation = get_chat_completions(conversation_reco)


                if check_moderation(user_input) == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break


                conversation_reco.append({"role": "user", "parts": [f"This is my user profile {response}"]})


                conversation_reco.append({"role": "model", "parts": [recommendation]})


                print("--"*20 + "Shop Assitant" + "--"*20)
                print(f'{recommendation}\n')


        else:
            conversation_reco.append({"role": "user", "parts": [user_input]})


            response_asst_reco = get_chat_completions(conversation_reco)

            if check_moderation(user_input) == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print("--"*20 + "Shop Assitant" + "--"*20)
            print(f'\n{response_asst_reco}\n')
            conversation.append({"role": "model", "parts": [response_asst_reco]})

# Run Simulation in Debug Mode

In [201]:
dialogue_mgmt_system(isDebug=True)

Initialising system, please wait....

System is ready 

----------------------------------------Shop Assitant----------------------------------------

    Hello, I am your laptop assistant. Please feel free to ask me about laptop recommendations. 

    You can tell me about your needs like 

    - what is the purpose of the laptop
    - Do you need heavy processing laptop
    - Do you need it for travel purposes, etc

    Please note: I do not take inputs to talk to about any specific brand as I am suppose to be non-biased. 
    Once I find a good recommendatio for you, then you can ask me about that laptop.

    Let's start!!!
    

----------------------------------------User----------------------------------------


 I am Mohit Dubey and I am looking for a laptop to code ML and AI. Please recommend


----------------------------------------Shop Assitant----------------------------------------

Hi Mohit!  Coding ML and AI requires a powerful laptop. To give you the best recommendations, I need to understand your specific needs. 

Could you tell me a little more about your coding projects? For example:

* **What kind of ML/AI projects are you working on?** (e.g., image recognition, natural language processing, deep learning)
* **Do you need to run large datasets or complex models?**
* **Do you need a lot of RAM or a powerful GPU?**
* **What is your budget for the laptop?**

Once I have a better understanding of your requirements, I can recommend the best laptop for your needs. 



{'result': 'No', 'reason': 'Missing keys'}

----------------------------------------User----------------------------------------


 Running NLP and Machine Learning tasks. 


----------------------------------------Shop Assitant----------------------------------------

Okay, so you're working on NLP and Machine Learning tasks. That's great!  To give you the best recommendations, I need a little more information. 

* **Do you work with large datasets or complex models?**  This will help determine the amount of RAM and processing power you need.
* **Do you need a dedicated GPU for accelerating your training and inference?**  If so, how powerful of a GPU do you need?
* **What is your budget for the laptop?**  This will help me narrow down the options.

Once I have a better understanding of your requirements, I can recommend the best laptop for your needs. 



{'result': 'No', 'reason': 'Missing keys'}

----------------------------------------User----------------------------------------


...

----------------------------------------Shop Assitant----------------------------------------

Okay, with a budget of less than 300,000 INR, we have some great options for a powerful laptop for your ML/AI work. 

Here's a breakdown of the key features you've mentioned and how they'll be reflected in the laptop recommendations:

* **GPU Intensity:**  Since you're running complex simulations, you'll need a dedicated GPU.  We'll aim for a high-end GPU, like an NVIDIA RTX 30 series or AMD Radeon RX 6000 series.
* **Display Quality:**  While not explicitly mentioned, a good display is important for working with data and code.  We'll aim for a high-resolution display with good color accuracy.
* **Portability:**  You've mentioned portability isn't a priority, so we can focus on laptops with larger screens and better performance.
* **Multitasking:**  Running complex simulations requires a lot of processing power and RAM.  We'll aim for a laptop with a powerful processor and at least 16GB of RA

 Which one do you recommend for running high GPU tasks. Portability is not a priority for me


----------------------------------------Shop Assitant----------------------------------------

Based on your preference for high GPU tasks and low priority on portability, I recommend the **Dell Precision 5550**. 

Here's why:

* **Powerful Graphics:** It features the NVIDIA Quadro graphics card, designed for professional workloads and demanding applications like 3D rendering, video editing, and CAD. 
* **High Performance:** The Xeon processor and 32GB RAM ensure smooth performance for multitasking and handling complex tasks.
* **Large Display:** The 15.6" OLED display provides ample screen real estate for your work.

While the ASUS ZenBook Pro also has a powerful NVIDIA RTX graphics card, the Dell Precision 5550's Quadro card is specifically optimized for professional applications, making it a better choice for your needs. 


----------------------------------------User----------------------------------------


 exit


----------------------------------------Shop Assitant----------------------------------------

Okay, I understand.  Let me know if you have any other questions. 




# Run Simulation

In [202]:
dialogue_mgmt_system()

Initialising system, please wait....

System is ready 

----------------------------------------Shop Assitant----------------------------------------

    Hello, I am your laptop assistant. Please feel free to ask me about laptop recommendations. 

    You can tell me about your needs like 

    - what is the purpose of the laptop
    - Do you need heavy processing laptop
    - Do you need it for travel purposes, etc

    Please note: I do not take inputs to talk to about any specific brand as I am suppose to be non-biased. 
    Once I find a good recommendatio for you, then you can ask me about that laptop.

    Let's start!!!
    

----------------------------------------User----------------------------------------


 I am Mohit Dubey and I am looking for a laptop to code ML and AI. Please recommend


----------------------------------------Shop Assitant----------------------------------------

Hi Mohit!  Coding ML and AI requires a powerful laptop. To give you the best recommendations, I need to understand your specific needs. 

Could you tell me a little more about your coding projects? For example:

* **What kind of ML/AI projects are you working on?** (e.g., image recognition, natural language processing, deep learning)
* **Do you need to run large datasets or complex models?**
* **Do you need a lot of RAM or a powerful GPU?**
* **What is your budget for the laptop?**

Once I have a better understanding of your requirements, I can recommend the best laptop for your needs. 


----------------------------------------User----------------------------------------


 Running NLP and Machine Learning tasks. 


----------------------------------------Shop Assitant----------------------------------------

Okay, so you're working on NLP and Machine Learning tasks. That's great!  To give you the best recommendations, I need a little more information. 

* **Do you work with large datasets or complex models?**  This will help determine the amount of RAM and processing power you need.
* **Do you need a dedicated GPU for your work?**  GPUs are essential for accelerating deep learning tasks.
* **What is your budget for the laptop?**  This will help me narrow down the options.

Once I have a better understanding of your requirements, I can recommend the best laptop for your needs. 


----------------------------------------User----------------------------------------


...

----------------------------------------Shop Assitant----------------------------------------

Okay, great!  A budget of 200,000 INR gives us some good options.  

Based on our conversation, here's what I'm thinking for your laptop profile:

* **GPU Intensity:** High (You'll need a dedicated GPU for complex simulations)
* **Display Quality:** Medium (Since you're focused on performance, a high-resolution display isn't as critical)
* **Portability:** Low (You're not prioritizing portability)
* **Multitasking:** High (You'll need a lot of RAM for complex simulations)
* **Processing Speed:** High (You need a powerful processor for complex simulations)
* **Budget:** 200000

I'm confident about these values based on your responses.  

Now, I can start recommending specific laptops that fit your profile.  Do you have any preferences for brands or specific features? 


----------------------------------------Shop Assitant----------------------------------------
Thank you for providing all the

 Which one do you recommend for running heavy CPU tasks. Portability is not a priority for me


----------------------------------------Shop Assitant----------------------------------------

Based on your preference for heavy CPU tasks and your budget, I recommend the **ASUS ZenBook Pro**. 

Here's why:

* **Powerful Processor:** It boasts an i9 Intel CPU with a 3.1 GHz clock speed, making it ideal for demanding tasks like video editing, 3D rendering, and software development.
* **Ample RAM:** With 64GB of RAM, you can run multiple applications simultaneously without experiencing lag or performance issues.
* **High-End Graphics:** While not the primary focus for CPU-intensive tasks, the NVIDIA RTX graphics card will still provide a smooth experience for general use and light gaming.

While the Dell XPS 15 also has a powerful i9 processor, the ASUS ZenBook Pro offers more RAM, which is crucial for handling heavy CPU workloads. 


----------------------------------------User----------------------------------------


 exit


----------------------------------------Shop Assitant----------------------------------------

Okay, I understand. Based on your updated preferences, I would recommend the **ASUS ZenBook Pro**. 

Here's why:

* **High Processing Speed:** It boasts an i9 Intel CPU, which is ideal for demanding CPU-intensive tasks.
* **Large RAM:** With 64GB of RAM, you'll have ample memory for multitasking and running resource-heavy applications.
* **Portability Not a Priority:** While it's still relatively lightweight, the ZenBook Pro's focus on performance over portability aligns with your needs.

The Dell XPS 15 is also a strong contender, but the ZenBook Pro's larger RAM capacity gives it an edge for heavy CPU tasks. 


